The goal of this notebook is to setup a global model training framework, where a single model is trained on all pc types.

In [ ]:
import os
from typing import Literal

from dotenv import load_dotenv
import mlflow
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error
from xgboost import XGBRegressor

from constants import processed_names
import constants.constants as cst
from constants.paths import PROCESSED_DATA_DIR
from src.utils.logger import logger
from src.utils.multivariate_model_training import (
    evaluate_and_log_model,
    optimize_xgboost_model,
)

load_dotenv()
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))

# 1. Load and Prepare Data 

We define a function to load data and separate features and target variable from the dataframe. There are different types of features:
- Target variable: `pc_price`.
- Meta features: `region`, `pc_type` and `date`. (Used for grouping and weighting but not as model features.)
- Numerical features: `pc_price_lag_*`, `pc_price_rolling_mean_*`, `regional_avg_price`, `regional_price_volaility_`, `price_deviation_from_regional_avg`, exogenous features like `bpa_capacity_loss_kt` and their lags (less lags than for target), and time features like `month_sin`, `month_cos` and raw `month` or `year`.
- Categorical binary features (can keep as is, tree based models handle $0$ and $1$):  `is_recycled`, `is_glass_filled`, `is_flame_retardant`.

In [ ]:
def load_and_prepare_data(
    group_by_pc_types: bool, horizon: int = 3
) -> tuple[pd.DataFrame, str, list[str]]:
    """Load processed data and separate features and target variable.

    Args:
        group_by_pc_types (bool): Whether PC prices are grouped by type.
        horizon (int, optional): Forecast horizon in months. Defaults to 3.

    Returns:
        tuple[pd.DataFrame, str, list[str]]: DataFrame, target column name,
        feature column names.
    """
    # Load processed data
    if group_by_pc_types:
        df = pd.read_csv(PROCESSED_DATA_DIR / f"multi_{horizon}m_grouped.csv")
    else:
        df = pd.read_csv(PROCESSED_DATA_DIR / f"multi_{horizon}m.csv")

    # Separate features and target
    target_col = processed_names.LONG_PC_PRICE
    meta_cols = [
        processed_names.LONG_DATE,
        processed_names.LONG_REGION,
        processed_names.LONG_PC_TYPE,
    ]
    feature_cols = [col for col in df.columns if col not in meta_cols + [target_col]]
    logger.info(f"Data loaded with {df.shape[0]} rows and {df.shape[1]} columns.")
    logger.info(f"Target: {target_col}. Features: {len(feature_cols)} columns.")

    return df, target_col, feature_cols

In [ ]:
df, target_col, feature_cols = load_and_prepare_data(group_by_pc_types=True, horizon=3)

# 2. Split Data

For data not grouped by PC types, we can use a standard train-validation-test split. However, since the data is imbalanced across different pc types, we need to ensure that the train set and the test set contain enough samples from each pc type.

In [ ]:
# Use for dataset not grouped by PC types
def adaptive_train_validation_test_split(
    df: pd.DataFrame,
    group_col: str = processed_names.LONG_PC_TYPE,
    target_test_ratio: float = 0.1,
    target_validation_ratio: float = 0.1,
    min_train_samples: int = 20,
    min_validation_samples: int = 5,
    min_test_samples: int = 5,
) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Adaptive temporal split that ensures minimum samples per group.

    If standard 80/20 split results in insufficient test samples for any group,
    adjusts split date to ensure minimum requirements.

    Args:
        df: Long format dataframe with 'date' column
        group_col: Column defining groups
        target_test_ratio: Desired test set size (default 0.1 = 10% of total data)
        target_validation_ratio: Desired validation set size (default 0.1 = 10%
                                 of total data)
        min_train_samples: Minimum training samples per group
        min_test_samples: Minimum test samples per group
        min_validation_samples: Minimum validation samples per group

    Returns:
        (train_df, validation_df,test_df) with sufficient samples per group
    """
    df = df.sort_values(processed_names.LONG_DATE).reset_index(drop=True)

    # Find the latest date that satisfies constraints
    dates = sorted(df[processed_names.LONG_DATE].unique())

    # Target PC types to check
    target_pc_types = [cst.REGULAR_PC_TYPE, cst.GREEN_PC_TYPE]

    for validation_split_date in reversed(dates):
        possible_test_dates = [date for date in dates if date > validation_split_date]
        for test_split_date in reversed(possible_test_dates):
            train = df[df[processed_names.LONG_DATE] < validation_split_date]
            validation = df[
                (df[processed_names.LONG_DATE] >= validation_split_date)
                & (df[processed_names.LONG_DATE] < test_split_date)
            ]
            test = df[df[processed_names.LONG_DATE] >= test_split_date]
            # Check if all groups meet minimum requirements
            valid = True
            for target_pc in target_pc_types:
                train_count = (train[group_col] == target_pc).sum()
                validation_count = (validation[group_col] == target_pc).sum()
                test_count = (test[group_col] == target_pc).sum()

                if (
                    train_count < min_train_samples
                    or validation_count < min_validation_samples
                    or test_count < min_test_samples
                ):
                    valid = False
                    break

            if valid:
                logger.info(
                    f"Found valid split at dates: "
                    f"validation {validation_split_date}, test {test_split_date}"
                )
                actual_test_ratio = len(test) / len(df)
                actual_validation_ratio = len(validation) / len(df)
                logger.info(f"Actual test set ratio: {actual_test_ratio:.2%}")
                logger.info(f"Desired test set ratio: {target_test_ratio:.2%}")
                logger.info(
                    f"Actual validation set ratio: {actual_validation_ratio:.2%}"
                )
                logger.info(
                    f"Desired validation set ratio: {target_validation_ratio:.2%}"
                )

                return train, validation, test

    # If no valid split found, fall back to target ratios
    logger.warning(
        f"No valid adaptive split found satisfying minimum sample constraints "
        f"(train≥{min_train_samples}, validation≥{min_validation_samples}, "
        f"test≥{min_test_samples}). Falling back to "
        f"{1 - target_test_ratio - target_validation_ratio:.0%}/"
        f"{target_validation_ratio:.0%}/{target_test_ratio:.0%} split."
    )

    # Use index-based splitting on unique dates (more reliable than quantile)
    dates = sorted(df[processed_names.LONG_DATE].unique())
    n_dates = len(dates)

    val_idx = int(n_dates * (1 - target_test_ratio - target_validation_ratio))
    test_idx = int(n_dates * (1 - target_test_ratio))

    split_date_validation = dates[val_idx]
    split_date_test = dates[test_idx]

    train = df[df[processed_names.LONG_DATE] < split_date_validation]
    validation = df[
        (df[processed_names.LONG_DATE] >= split_date_validation)
        & (df[processed_names.LONG_DATE] < split_date_test)
    ]
    test = df[df[processed_names.LONG_DATE] >= split_date_test]

    # Log warning about groups that don't meet requirements
    for target_pc in target_pc_types:
        train_count = (train[group_col] == target_pc).sum()
        val_count = (validation[group_col] == target_pc).sum()
        test_count = (test[group_col] == target_pc).sum()

        if (
            train_count < min_train_samples
            or val_count < min_validation_samples
            or test_count < min_test_samples
        ):
            logger.warning(
                f"PC type '{target_pc}' has insufficient samples in fallback split: "
                f"train={train_count} (min={min_train_samples}), "
                f"val={val_count} (min={min_validation_samples}), "
                f"test={test_count} (min={min_test_samples})"
            )

    return train, validation, test

In [ ]:
# train, validation, test = adaptive_train_validation_test_split(
#     df=df, group_col=processed_names.LONG_PC_TYPE
# )

For data grouped by PC types, because the data is imbalanced across different pc types, we only perform a train-test split (no validation set). We need to ensure that the train set and the test set contain enough samples from each pc type. This is a problem especially for rare pc types (`gf20` notably). To do this, we use a function that performs an adaptive train-test split, ensuring that each pc type is represented in both sets with a minimum number of samples.

In [ ]:
# Use for data grouped by PC types
def adaptive_train_test_split(
    df: pd.DataFrame,
    group_col: str = processed_names.LONG_PC_TYPE,
    target_test_ratio: float = 0.2,
    min_train_samples: int = 20,
    min_test_samples: int = 5,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Adaptive temporal split that ensures minimum samples per group.

    If standard 80/20 split results in insufficient test samples for any group,
    adjusts split date to ensure minimum requirements.

    Args:
        df: Long format dataframe with 'date' column
        group_col: Column defining groups
        target_test_ratio: Desired test set size (default 0.2 = 20%)
        min_train_samples: Minimum training samples per group
        min_test_samples: Minimum test samples per group

    Returns:
        (train_df, test_df) with sufficient samples per group
    """
    df = df.sort_values(processed_names.LONG_DATE).reset_index(drop=True)

    # Find the latest date that satisfies constraints
    dates = sorted(df[processed_names.LONG_DATE].unique())

    for split_date in reversed(dates):
        train = df[df[processed_names.LONG_DATE] < split_date]
        test = df[df[processed_names.LONG_DATE] >= split_date]
        # Check if all groups meet minimum requirements
        valid = True
        target_pc_types = [
            pc_type
            for pc_type in df[group_col].unique()
            if pc_type in cst.PCType._value2member_map_
        ]
        for target_pc in target_pc_types:
            train_count = (train[group_col] == target_pc).sum()
            test_count = (test[group_col] == target_pc).sum()

            if train_count < min_train_samples or test_count < min_test_samples:
                valid = False
                break

        if valid:
            logger.info(f"Found valid split at date: {split_date}")
            actual_test_ratio = len(test) / len(df)
            logger.info(f"Actual test set ratio: {actual_test_ratio:.2%}")
            logger.info(f"Desired test set ratio: {target_test_ratio:.2%}")

            return train, test

    # If no valid split found, warn user
    logger.warning(
        "Could not find a split date satisfying minimum sample requirements "
        "for all groups. Falling back to target test ratio split."
    )

    # Fallback: use target ratio
    n_test = int(len(dates) * target_test_ratio)
    split_date = dates[-n_test]
    train = df[df[processed_names.LONG_DATE] < split_date]
    test = df[df[processed_names.LONG_DATE] >= split_date]

    # Log warning about groups that don't meet requirements
    for target_pc in target_pc_types:
        train_count = (train[group_col] == target_pc).sum()
        test_count = (test[group_col] == target_pc).sum()

        if train_count < min_train_samples or test_count < min_test_samples:
            logger.warning(
                f"PC type '{target_pc}' has insufficient samples in fallback split: "
                f"train={train_count} (min={min_train_samples}), "
                f"test={test_count} (min={min_test_samples})"
            )

    return train, test

In [ ]:
# train, test = adaptive_train_test_split(
#     df=df,
#     group_col=processed_names.LONG_PC_TYPE,
#     min_train_samples=19,
#     min_test_samples=5,
# )

# 3. Prepare Features and Target

In [ ]:
def prepare_training_data(
    train_df: pd.DataFrame,
    validation_df: pd.DataFrame | None,
    test_df: pd.DataFrame,
    feature_cols: list[str],
    target_col: str,
    horizon: int,
) -> tuple[
    np.ndarray,
    np.ndarray | None,
    np.ndarray,
    np.ndarray,
    np.ndarray | None,
    np.ndarray,
    pd.DataFrame,
    pd.DataFrame | None,
    pd.DataFrame,
]:
    """Prepare features and target variable for training.

    To align the target variable with the features, the target is shifted
    by -horizon months within each group defined by region and pc_type.

    Args:
        train_df (pd.DataFrame): Training dataframe.
        validation_df (pd.DataFrame | None): Validation dataframe.
        test_df (pd.DataFrame): Testing dataframe.
        feature_cols (list[str]): List of feature column names.
        target_col (str): Target column name.
        horizon (int): Forecast horizon in months.

    Returns:
        tuple[np.ndarray, np.ndarray | None, np.ndarray, np.ndarray, np.ndarray | None,
        np.ndarray, pd.DataFrame, pd.DataFrame | None, pd.DataFrame]:
            X_train, y_train, X_validation, y_validation, X_test, y_test,
            aligned training, validation and testing dataframes
    """
    # Target (shift by -horizon to align with features)
    # Group by region and pc_type to shift correctly
    train_df["target"] = train_df.groupby(
        [processed_names.LONG_REGION, processed_names.LONG_PC_TYPE]
    )[target_col].shift(-horizon)
    if validation_df is not None:
        validation_df["target"] = validation_df.groupby(
            [processed_names.LONG_REGION, processed_names.LONG_PC_TYPE]
        )[target_col].shift(-horizon)
    test_df["target"] = test_df.groupby(
        [processed_names.LONG_REGION, processed_names.LONG_PC_TYPE]
    )[target_col].shift(-horizon)

    # Drop rows with NaN in target (due to shifting)
    train_mask = ~train_df["target"].isna()
    if validation_df is not None:
        validation_mask = ~validation_df["target"].isna()
    test_mask = ~test_df["target"].isna()

    X_train = train_df.loc[train_mask, feature_cols].values
    y_train = train_df.loc[train_mask, "target"].values
    train_df_aligned = train_df[train_mask].copy()
    logger.info(f"Training data prepared with {X_train.shape[0]} samples.")

    if validation_df is not None:
        X_validation = validation_df.loc[validation_mask, feature_cols].values
        y_validation = validation_df.loc[validation_mask, "target"].values
        validation_df_aligned = validation_df[validation_mask].copy()
        logger.info(f"Validation data prepared with {X_validation.shape[0]} samples.")

    X_test = test_df.loc[test_mask, feature_cols].values
    y_test = test_df.loc[test_mask, "target"].values
    test_df_aligned = test_df[test_mask].copy()
    logger.info(f"Testing data prepared with {X_test.shape[0]} samples.")

    if validation_df is not None:
        return (
            X_train,
            y_train,
            X_validation,
            y_validation,
            X_test,
            y_test,
            train_df_aligned,
            validation_df_aligned,
            test_df_aligned,
        )

    return X_train, y_train, X_test, y_test, train_df_aligned, test_df_aligned

In [ ]:
# X_train, y_train, X_test, y_test, train_df_aligned, test_df_aligned = (
#     prepare_training_data(
#         train_df=train,
#         validation_df=None,
#         test_df=test,
#         feature_cols=feature_cols,
#         target_col=target_col,
#         horizon=3,
#     )
# )

# (
#     X_train,
#     y_train,
#     X_validation,
#     y_validation,
#     X_test,
#     y_test,
#     train_df_aligned,
#     validation_df_aligned,
#     test_df_aligned,
# ) = prepare_training_data(
#     train_df=train,
#     validation_df=validation,
#     test_df=test,
#     feature_cols=feature_cols,
#     target_col=target_col,
#     horizon=3,
# )

# 4. Compute Sample Weights

Because the data is imbalanced across different pc types, we compute sample weights to give more importance to under-represented pc types during model training. This helps the model to learn better representations for these rare pc types. Without this, the model might be biased towards the more common pc types, leading to poor performance on the rare ones. The global performance metric might be good, but the performance on rare pc types would be bad.

Additionally, we can also weight samples based on region: we are only concerned about performance in Europe, so we can give more weight to samples from this region. We keep pc types from all regions in the training set to have more data, but we want to prioritize performance on European pc types.

In [ ]:
def compute_sample_weights(
    df: pd.DataFrame,
    group_col: str,
    region_col: str,
    target_region: str,
    method: Literal["inverse_frequency", "sqrt_inverse", "balanced"] = "balanced",
    region_weight_multiplier: float = 1.5,
) -> pd.Series:
    """Compute sample weights based on group frequency and region.

    Args:
        df (pd.DataFrame): Dataframe containing the data.
        group_col (str): Column name for grouping (e.g., pc_type).
        region_col (str): Column name for region.
        target_region (str): Region to prioritize.
        method (Literal["inverse_frequency", "sqrt_inverse", "balanced"], optional):
            Method to compute weights. Defaults to "balanced".
        region_weight_multiplier (float, optional): Multiplier for weights of the
            target region. Defaults to 1.5.

    Returns:
        pd.Series: Sample weights for each row in the dataframe.
    """
    # Validate method
    if method not in ["inverse_frequency", "sqrt_inverse", "balanced"]:
        raise ValueError(
            f"Invalid method: {method}. Choose from 'inverse_frequency', "
            "'sqrt_inverse', 'balanced'."
        )

    group_counts = df[group_col].value_counts()
    n_samples = len(df)
    n_groups = len(group_counts)

    # Simple inverse frequency weights (1/count)
    if method == "inverse_frequency":
        weights_map = {group: 1.0 / count for group, count in group_counts.items()}
        weights = df[group_col].map(weights_map).values

    # Square root of inverse frequency weights
    # Less aggressive than inverse_frequency
    elif method == "sqrt_inverse":
        weights_map = {
            group: 1.0 / np.sqrt(count) for group, count in group_counts.items()
        }
        weights = df[group_col].map(weights_map).values

    elif method == "balanced":
        # Sklearn-style balanced weights: n_samples / (n_groups * count)
        # Normalized so sum(weights) ≈ n_samples (maintains effective sample size)
        weights_map = {
            group: n_samples / (n_groups * count)
            for group, count in group_counts.items()
        }
        weights = df[group_col].map(weights_map).values

        # Apply region multiplier
    region_multiplier = np.where(
        df[region_col] == target_region,
        region_weight_multiplier,  # 1.5× weight for Europe
        1.0,  # 1.0× weight for Asia
    )
    weights = weights * region_multiplier

    # Renormalize to maintain effective sample size
    weights = weights * len(df) / weights.sum()

    return weights

# 5. Define evaluation metrics

For model evaluation, we will use the Mean Absolute Percentage Error (MAPE) as our primary metric. MAPE is particularly useful in this context because it provides a normalized measure of prediction accuracy, allowing us to assess how well our model performs across different pc types and price ranges. However, using just a global MAPE can be misleading due to the imbalanced nature of the dataset. To address this, we will also compute a weighted MAPE, where each pc type's contribution to the overall metric is weighted inversely proportional to its frequency in the dataset. This approach ensures that the model's performance on rare pc types is adequately represented in the evaluation, preventing the model from being overly optimized for the more common pc types at the expense of the rare ones. We have $3$ metrics in total:
- Global MAPE: Overall MAPE across all samples.
- Weighted MAPE: MAPE computed with pc type weights.
- Per pc type MAPE: MAPE computed for each pc type separately.

In [ ]:
def compute_performance_metrics(
    y_true: np.ndarray,
    y_pred: np.ndarray,
    pc_types: pd.Series,
    weights: np.ndarray = None,
) -> dict[str, float]:
    """Compute performance metrics for the model.

    Three performance metrics are computed:
    - Global MAPE: Mean Absolute Percentage Error across all samples.
    - Weighted MAPE: MAPE computed with sample weights.
    - Per pc type MAPE: MAPE computed for each pc type separately.

    Args:
        y_true (np.ndarray): True target values.
        y_pred (np.ndarray): Predicted target values.
        pc_types (pd.Series): PC types of the true target values.
        weights (np.ndarray, optional): Sample weights. Defaults to None.

    Returns:
        dict[str, float]: Dictionary containing the computed performance metrics.
    """
    performance_metrics = {}

    # Global MAPE
    global_mape = mean_absolute_percentage_error(y_true, y_pred)
    performance_metrics[cst.GLOBAL_MAPE] = global_mape

    # Weighted MAPE
    if weights is not None:
        weighted_mape = mean_absolute_percentage_error(
            y_true, y_pred, sample_weight=weights
        )
        performance_metrics[cst.WEIGHTED_MAPE] = weighted_mape

    # Per PC type MAPE
    for pc_type in pc_types.unique():
        pc_type_mask = pc_types == pc_type
        y_pc_specific_true = y_true[pc_type_mask]
        y_pc_specific_pred = y_pred[pc_type_mask]
        pc_specific_mape = mean_absolute_percentage_error(
            y_pc_specific_true, y_pc_specific_pred
        )
        performance_metrics[f"{pc_type}_MAPE"] = pc_specific_mape

    return performance_metrics

# 6. Train Global Model

We train a single global model on all pc types using the computed sample weights, and log it to MLflow. This gives us the following training pipeline.

In [ ]:
def train_global_model(
    group_by_pc_types: bool,
    horizon: int,
    target_test_ratio: float,
    target_validation_ratio: float,
    min_train_samples: int,
    min_validation_samples: int,
    min_test_samples: int,
    weighting_method: Literal["inverse_frequency", "sqrt_inverse", "balanced"],
    model_type: Literal["xgboost", "random_forest", "lightgbm", "catboost", "tft"],
    hyperparameter_grid: dict | None,
    mlflow_run_name: str,
    n_trials: int,
) -> None:
    """Train a global model on all PC types and log it to MLflow.

    Args:
        group_by_pc_types (bool): Whether PC prices are grouped by type.
        horizon (int): Forecast horizon in months.
        target_test_ratio (float): Desired test set size.
        target_validation_ratio (float): Desired validation set size.
        min_train_samples (int): Minimum training samples per group.
        min_validation_samples (int): Minimum validation samples per group.
        min_test_samples (int): Minimum test samples per group.
        weighting_method (Literal): Method to compute sample weights.
        model_type (Literal): Type of model to train.
        hyperparameter_grid (dict | None): Hyperparameter grid for optimization.
        mlflow_run_name (str): Name for the MLflow run.
        n_trials (int): Number of trials for hyperparameter optimization.
    """
    # 1. Load and prepare data
    df, target_col, feature_cols = load_and_prepare_data(
        group_by_pc_types=group_by_pc_types, horizon=horizon
    )

    # 2. Split data into training and testing sets
    if group_by_pc_types:
        train_df, test_df = adaptive_train_test_split(
            df=df,
            group_col=processed_names.LONG_PC_TYPE,
            target_test_ratio=target_test_ratio,
            min_train_samples=min_train_samples,
            min_test_samples=min_test_samples,
        )
    else:
        train_df, validation_df, test_df = adaptive_train_validation_test_split(
            df=df,
            group_col=processed_names.LONG_PC_TYPE,
            target_test_ratio=target_test_ratio,
            target_validation_ratio=target_validation_ratio,
            min_train_samples=min_train_samples,
            min_validation_samples=min_validation_samples,
            min_test_samples=min_test_samples,
        )

    # 3. Prepare training and testing data
    if group_by_pc_types:
        X_train, y_train, X_test, y_test, train_df_aligned, test_df_aligned = (
            prepare_training_data(
                train_df=train_df,
                validation_df=None,
                test_df=test_df,
                feature_cols=feature_cols,
                target_col=target_col,
                horizon=horizon,
            )
        )
    else:
        (
            X_train,
            y_train,
            X_validation,
            y_validation,
            X_test,
            y_test,
            train_df_aligned,
            validation_df_aligned,
            test_df_aligned,
        ) = prepare_training_data(
            train_df=train_df,
            validation_df=validation_df,
            test_df=test_df,
            feature_cols=feature_cols,
            target_col=target_col,
            horizon=horizon,
        )

    # 4. Compute sample weights for train and test
    train_sample_weights = compute_sample_weights(
        df=train_df_aligned,
        group_col=processed_names.LONG_PC_TYPE,
        region_col=processed_names.LONG_REGION,
        target_region=cst.EUROPE,
        method=weighting_method,
    )

    if not group_by_pc_types:
        validation_sample_weights = compute_sample_weights(
            df=validation_df_aligned,
            group_col=processed_names.LONG_PC_TYPE,
            region_col=processed_names.LONG_REGION,
            target_region=cst.EUROPE,
            method=weighting_method,
        )

    test_sample_weights = compute_sample_weights(
        df=test_df_aligned,
        group_col=processed_names.LONG_PC_TYPE,
        region_col=processed_names.LONG_REGION,
        target_region=cst.EUROPE,
        method=weighting_method,
    )

    # 5. Train model
    if model_type == "xgboost":
        if group_by_pc_types:
            best_params = optimize_xgboost_model(
                X_train=X_train,
                y_train=y_train,
                X_test=X_test,
                y_test=y_test,
                weights=test_sample_weights,
                hyperparameter_grid=hyperparameter_grid,
                pc_types=test_df_aligned[processed_names.LONG_PC_TYPE],
                n_trials=n_trials,
            )
        else:
            best_params = optimize_xgboost_model(
                X_train=X_train,
                y_train=y_train,
                X_test=X_validation,
                y_test=y_validation,
                weights=validation_sample_weights,
                hyperparameter_grid=hyperparameter_grid,
                pc_types=validation_df_aligned[processed_names.LONG_PC_TYPE],
                n_trials=n_trials,
            )
        eval_model = XGBRegressor(**best_params)
        pred_model = XGBRegressor(**best_params)

        # 6. Evaluate and log model
        evaluate_and_log_model(
            eval_model=eval_model,
            pred_model=pred_model,
            best_params=best_params,
            mlflow_run_name=mlflow_run_name,
            model_type=model_type,
            horizon=horizon,
            group_by_pc_types=group_by_pc_types,
            weighting_method=weighting_method,
            X_train=X_train,
            y_train=y_train,
            X_validation=X_validation if not group_by_pc_types else None,
            y_validation=y_validation if not group_by_pc_types else None,
            X_test=X_test,
            y_test=y_test,
            train_df_aligned=train_df_aligned,
            validation_df_aligned=validation_df_aligned
            if not group_by_pc_types
            else None,
            test_df_aligned=test_df_aligned,
            train_sample_weights=train_sample_weights,
            test_sample_weights=test_sample_weights,
        )

    elif model_type == "random_forest":
        # Placeholder for random forest implementation
        pass

    elif model_type == "lightgbm":
        # Placeholder for LightGBM implementation
        pass

    elif model_type == "catboost":
        # Placeholder for CatBoost implementation
        pass

    elif model_type == "tft":
        # Placeholder for TFT implementation
        pass
    else:
        raise NotImplementedError(f"Model type '{model_type}' not implemented.")

In [ ]:
train_global_model(
    group_by_pc_types=False,
    horizon=3,
    target_test_ratio=0.2,
    target_validation_ratio=0.1,
    min_train_samples=19,
    min_test_samples=50,
    min_validation_samples=50,
    weighting_method="balanced",
    model_type="xgboost",
    hyperparameter_grid=None,
    mlflow_run_name="global_xgboost_model_3m_horizon_no_grouping_v3",
    n_trials=500,
)

In [ ]:
mlflow.search_runs()